# Student ID: R00182510


# Assignment Part 1 - 2D World 


In [1]:
##imports the parent directory - aima-python
import sys
sys.path.insert(0,'../..')

##other libraries
from agents import *
from notebook import psource
from ipythonblocks import BlockGrid
from search import *
import random
import math
import collections
from itertools import combinations
from itertools import chain,cycle

# DESIGN AND MECHANICS OF THE 2D WORLD
The 2D world modelled here is a grid of size 6x6 tiles, that will be used to demonstrate the ability of three different agents in an Environment. The Environment is a 'Shopping Mall' in which an agent,  a 'Woman' is trying to locate her friend and pick her up. The final target of the agent is to pick up her 'Friend'. The Environment also includes obstacles that make it difficult for the agent to achieve her target. The obstacle in the current world is 'Fire'. For every tile that contains Fire, its adjacent tiles are indicated as 'Heat'. 
The placements of the obstacles (Fire) and the goal (Friend) is completely randomised for all the agent-environment instances. The world also has walls in it to provide a boundary within which the agent can play the game.

The three different agent types that were implemented are:
<b>
- Simple Reflex Agent
- Model Based Agent
- Goal Based Agent
</b>
Additionally, a small discussion on how a utility-based agent would perform in the same environment is also included.

The behavior of each agent varies based on the agent program implementaion of the agent function. Each of these agents will have its own abilities to map its percepts to its actions.

The Environment can be randomised with respect to its size, positioning the obstacles and goal of the agent. The agent game is complete once she picks up her friend. In all the instances, different colour codes indicate the Environment and the objects in it. 


The colour coding is as follows.
<b>
- Agent - Purple <br>
- Walls - Black<br>
- Tiles without any objects - Green<br>
- Tiles with obstacles, "Fire" - Red <br>
- Tiles adjacent to the fire tiles, percieved as "Heat" - Orange <br>
- Friend location - Yellow <br>
</b>


Note : The core elements of the following code was referred and/or used from the AIMA data repositary.Hence, this cde file and the supporting file needs to be executed in the AIMA directory along with the AIMA files.

# 2D Environment

In [2]:
########### The below code was partially referenced from the file agents.ipynb and agents.py in the aima repository

class Fire(Thing):
    pass

class Heat(Thing):
    pass

class Friend(Thing):
    
    def __eq__(self, rhs):
        return rhs.__class__ == Friend
    pass

class Bump(Thing):
    pass


## Agent based Environment 


class Shopping_Mall2D(XYEnvironment):

    fire_probability = 0.1  # Probability to spawn fire in a location. (From Chapter 7.2)

    def __init__(self, agent_program, width=5, height=5):
        super().__init__(width, height)
        self.visible = False
        self.init_world(agent_program)

    def init_world(self, program):
        """Spawn items in the world based on probabilities"""

        "WALLS"
        self.add_walls()

        "FIRE"
        for x in range(self.x_start, self.x_end):
            for y in range(self.y_start, self.y_end):
                if (x == 1) & (y == 1):
                    continue
                if random.random() < self.fire_probability:
                    self.add_thing(Fire(), (x, y), True)
                    self.add_thing(Heat(), (x - 1, y), True)
                    self.add_thing(Heat(), (x, y - 1), True)
                    self.add_thing(Heat(), (x + 1, y), True)
                    self.add_thing(Heat(), (x, y + 1), True)

        "FRIEND"
        self.add_thing(Friend(), self.random_location_inbounds(exclude=[(1, 1)]), True)


    def get_world(self, show_walls=True):
        """Return the items in the world"""
        result = []
        x_start, y_start = (0, 0) if show_walls else (1, 1)

        if show_walls:
            x_end, y_end = self.width, self.height
        else:
            x_end, y_end = self.width - 1, self.height - 1

        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    def percepts_from(self, agent, location, tclass=Thing):
        """Return percepts from a given location,
        and replaces some items with percepts from chapter 7."""
        thing_percepts = {
            Friend: Friend(),
            Wall: Bump(),
            Fire: Fire(),
            Heat: Heat()}

        """Agents don't need to get their percepts"""
        thing_percepts[agent.__class__] = None

        result = [thing_percepts.get(thing.__class__, thing) for thing in self.things
                  if thing.location == location and isinstance(thing, tclass)]
        return result if len(result) else [None]

    def percept(self, agent):
        """Return things in adjacent (not diagonal) cells of the agent.
        Result format: [Left, Right, Up, Down, Center / Current location]"""
        x, y = agent.location
        result = []
        result.append(self.percepts_from(agent, (x - 1, y)))
        result.append(self.percepts_from(agent, (x + 1, y)))
        result.append(self.percepts_from(agent, (x, y - 1)))
        result.append(self.percepts_from(agent, (x, y + 1)))
        result.append(self.percepts_from(agent, (x, y)))
        
        return (result,agent.location,agent.direction.direction,agent.visited)
    
    def execute_action(self, agent, action):
        """Modify the state of the environment based on the agent's actions.
        Performance score taken directly out of the book."""

        if self.in_danger(agent):
            return

        agent.bump = False
        if action == 'TurnRight':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.direction += Direction.R
            agent.performance -= 1
        elif action == 'TurnLeft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.direction += Direction.L
            agent.performance -= 1
        elif action == 'Forward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
            agent.performance -= 1
            agent.visited.add(agent.location)
        elif action == 'Pick':
            things = [thing for thing in self.list_things_at(agent.location)
                      if agent.can_pick(thing)]
            if len(things):
                print("Picking up", things[0].__class__.__name__)
                if len(things):
                    agent.holding.append(things[0])
                    self.delete_thing(things[0])
                    agent.performance += 1000
                    print('Agent Performance score : ',agent.performance)
                    

    def in_danger(self, agent):
        """Check if Rachel is in danger (Fire), if she is, kill her"""
        for thing in self.list_things_at(agent.location):
            if isinstance(thing, Fire):
                agent.alive = False
                agent.performance -= 1000
                agent.killed_by = thing.__class__.__name__
                return True
        return False

    def is_done(self):
        """The game is over when Rachel is killed
        or if she picks up her friend."""
        #woman = [agent for agent in self.agents]
        #if woman[0].alive:
        if any(agent.is_alive() for agent in self.agents):
            goal_completed = not any(isinstance(thing, Friend) for thing in self.things)
            if goal_completed:
                print("Woman completed her goal{}."
                  .format(" with Friend [+1000]!" if Friend() not in self.things else " without Friend [+0]"))
                return True
            else:
                return False
        else:
            dead_agents = not any(agent.is_alive() for agent in self.agents)
            print("Death by Fire [-1000].")
            print('Agent Performance score : ',0)
            return True

    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end-1),
                    random.randint(self.y_start, self.y_end-1))
        if exclude is not None:
            while(location == exclude):
                location = (random.randint(self.x_start, self.x_end-1),
                            random.randint(self.y_start, self.y_end-1))
        return location
    
    def update(self, delay=1):
        sleep(delay)
        if self.visible:
            self.conceal()
            self.reveal()
        else:
            self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_grid(w.get_world())
        self.grid.show()
        self.visible = True

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))

# Simple Reflex Agent

A simple reflex agent will act on an environment based on the current percepts only, without including any knowledge on its percept history nor with an intention to reach the goal. The agent "program" function will interpret the current percept from the environment by matching against a set of rules and the "execute action" function returns the corresponding action to be executed by the host environment.The PEAS description for the simple reflex agent is given below.

### PEAS Description:
* <b>Performance measure:</b> 

    The game is complete once the agent picks up her friend or the agent is killed by Fire. The performamnce score is determined as given below.
    

    - Picking up her friend - +1000
    - Trapped in Fire - +0
    - Each step moved forward into a tile - (-1)
    
* <b>Environment:</b><br>
A 6X6 grid of tiles. The initial position for an agent to start is always [1,1], and will be facing initially to the Right. The tiles for the poitioning of the goal and the obstacle Fire, are randomly chosen based on a uniform probability distribution from all the tiles, excluding the initial position of the agent.


* <b>Actuators: </b><br>
With respect to the orientation of the agent, she will either be able to "TurnRight" or "TurnLeft" by 90 degrees and will be able to move "Forward" in whichever direction she is currently facing. If the Forward step is towards a bump/ wall tile, the agent position doesn't change. If the agent is trapped in a tile with Fire, she is dead. If the agent is in the location of her 
friend, she can "Pick" her friend which is her final goal.


* <b>Sensors: </b>
  - In friend location, the agent perceives "Friend". The simple reflex agent program will then initiate an action to pick up the friend. 
  - "TurnLeft", "TurnRight" and move "Forward" actions are executed at random.


### ADVANTAGES OF SIMPLE REFLEX AGENT:
* It is simple with not much knowledge on the agent's previous actions.
* Such a randomized agent behaviour can be a good rational in multi agent environment[1].

### DISADVANTAGES OF SIMPLE REFLEX AGENT
* As the agent type is too simple, the intelligence associated with such an agent is very limited. The agent will not be able to act correctly as only the current percept is made available to it.
* In single agent environments, this agent type is not a good rational[1].
* Due to the completely randomized behaviour, the agent would take a very long time to complete the game.

### PERFORMANCE IN THE CURRENT 2D WORLD
The overall performance of the agent type was calcualted, by recording the performance score in the agent-environment game for 5 runs, and then by calcualting the average performance score from these 5 runs. 

<table> 
    <tr>
        <td> </td>
        <td> Run 1 <br> Performance </td>
        <td> Run 2 <br> Performance </td>
        <td> Run 3 <br> Performance </td>
        <td> Run 4 <br> Performance </td>
        <td> Run 5 <br> Performance </td>
        <td> Average performance </td>
    </tr>
    <tr>
        <td> Performance<br>score: </td>
        <td> 856 </td>
        <td> 999 </td>
        <td> 0 </td>
        <td> 0 </td>
        <td> 943 </td>
        <td> 560 </td>            
   </tr>
</table>
    
   <t><b> Average performance score : 560 </b></t>
        

### PERFORMANCE IN EXTENDED ENVIRONMENTS

If the environment size is increased, the performance of a simple reflex agent would further drop down as its randomized behaviour might make it difficult for the agent to achieve its goal. The amount of time taken by the agent to complete the game would also be much higher. 

If the probability of a tile having the obstacle Fire, is increased, then the probability of the agent being trapped in the Fire increases with it, thus decreasing the performance significatly. 


In [3]:
### Simple reflex agent

color = {"Fire": (255, 83, 71),
        "Friend": (240, 255, 123),
        "Heat": (230, 115, 40),
        "Simple_Reflex_Woman": (202, 0, 112),
        "Wall": (44, 53, 57)
        }
  
class Simple_Reflex_Woman(Agent):
    holding = []
    killed_by = ""
    direction = Direction("right")
    visited = {(1,1)}

    def can_pick(self, thing):
        """Woman can only pickup friend"""
        return thing.__class__ == Friend
    
    
def program(percepts):
    '''Returns an action based on it's percepts'''
    
    percept, current_loc, orientation, visited = percepts
    
    for p in percept[4]:
        if isinstance(p,Friend):
            print('Ín Friend location')
            return 'Pick'
    
    choice = random.choice((1,2,3,4))
        
    if choice == 1:
        return 'TurnRight'
    elif choice == 2:
        return 'TurnLeft'
    else:
        return 'Forward'


w = Shopping_Mall2D(program, 6, 6)   
Rachel = Simple_Reflex_Woman(program)
w.add_thing(Rachel, (1, 1), True)
grid = BlockGrid(w.width, w.height, fill=(123, 234, 123))


def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]
                

def run(steps=50, delay=1):
    """Run the Environment for given number of time steps,
        but update the GUI too."""
    for step in range(steps):
        global grid, w
        draw_grid(w.get_world())
        grid.show()
        if w.is_done():
            break
        w.step()

        

In [4]:
run(20)

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Simple_Reflex_Woman decided to move rightwards at location: (1, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Death by Fire [-1000].
Agent Performance score :  0


# Model Based Agent

A model based agent will act on a partially observable environment, by <i>keeping track of a part of the world that it can't see now </i>[Reference 1] .

This means, tat the agent will act on an environment based on the current percepts and will additionally use its knowledge based on its percept history or its previous actions in the environment to make its decisions.<br>


<b>In the current 2D world, the percept history includes the tiles that has already been visited by the agent. With this information, the intention is to prevent the agent from moving into the tile that has already been visited.</b><br>


The agent "program" function will interpret the current percept from the environment and will also percieve if the tile to be moving into, has already been visited. If the proposed location has already been visited, then the agent program will propose a different action. The "execute action" function, then executes the corresponding action in the host environment.


The PEAS description for the model based agent is given below. The main variation of the 2D mechanics when comparing agents is in the Sensors and the Actuators part of the PEAS Description.



### PEAS Description:
* <b>Performance measure:</b> 

    Same as mentioned for the simple reflex agent.<br>
    
    
* <b>Environment:</b><br>

    Same as mentioned for the simple reflex agent.<br>
    

* <b>Actuators: </b><br>
With respect to the orientation of the agent, she will either be able to "TurnRight" or "TurnLeft" by 90 degrees and will be able to move "Forward" in whichever direction she is currently facing. If the Forward step is towards a bump/ wall tile, the agent position doesn't change. If the agent is trapped in a tile with Fire, she is dead. If the agent is in the location of her 
friend, she can "Pick" her friend which is her final goal.

  <b> Whenever the "Forward" action is executed, the agent marks the tile from which it moved as "visited".</b>


* <b>Sensors: </b>
  - In friend location, the agent perceives "Friend". The model based agent program will then initiate an action to pick up the friend. 
  - To determine if the "Forward" action is valid, the model based agent will look into its percept history and determine if the location is already visited. If it has been visited, then the move "Forward" action is considered invalid and the agent program, will be prompted for alternative action choices. If the proposed location tile has not been visited yet, then the move "Forward" action by the agent is considered a valid action.
  - "TurnLeft", "TurnRight" actions are executed at random.


### ADVANTAGES OF MODEL BASED AGENT
* As an added advantage to the simple reflex agent, the model based agent will prevent itself from moving into the same tile again and again. This results in increasing the performance with respect to moving between tiles.
* Another benefit of using a model based agent is that, it will take a much lesser time to reach its goal as the same steps are not repeated.
* Such an agent behaviour can be a good rational in both single and multi agent environments.

### DISADVANTAGES OF MODEL BASED AGENT
* As the agent is restricted from moving to the tile already visited, the risk of an agent being trapped in a particular cell is higher, if all its adjacent tiles are in the visited list. Thus the game will never converge to completion.
* The other risk is that, the probability of the agent being trapped in a "Fire" tile increases, as it will be forced to move into the Fire tile if all its ajacent tiles are in the visited list.


### PERFORMANCE IN THE CURRENT 2D WORLD
The overall performance of the agent type was calcualted, by recording the performance score in the agent-environment game for 5 runs, and then by calcualting the average performance score from these 5 runs. 

<table> 
    <tr>
        <td> </td>
        <td> Run 1 <br> Performance </td>
        <td> Run 2 <br> Performance </td>
        <td> Run 3 <br> Performance </td>
        <td> Run 4 <br> Performance </td>
        <td> Run 5 <br> Performance </td>
        <td> Average performance </td>
    </tr>
    <tr>
        <td> Performance<br>score: </td>
        <td> 0 </td>
        <td> 945 </td>
        <td> 0 </td>
        <td> 944 </td>
        <td> 'Agent Trapped<br> (no game completion) ' </td>
        <td> 472 </td>            
   </tr>
</table>
    
   <t><b> Average performance score : 472   </b></t>
        

### PERFORMANCE IN EXTENDED ENVIRONMENTS

If the environment size is increased, the performance of a model based agent would proportionally increase/decrease with it. The probability of the agent to reach the goal or to be trapped in Fire will be more influenced by random positioning of the tiles containing "Friend" or "Fire". As the visited tiles are not visited again, whichever(Friend or Fire) is closer to the agent's initial position, will be reached earlier.

If the probability of a tile having the obstacle Fire, is increased, then the probability of the agent being trapped in the Fire increases, thus decreasing the performance significatly. 


In [5]:
### Model Based Agent
    
color = {"Fire": (255, 83, 71),
        "Friend": (240, 255, 123),
        "Heat": (230, 115, 40),
        "Model_Based_Woman": (202, 0, 112),
        "Wall": (44, 53, 57)
        }

class Model_Based_Woman(Agent):
    holding = []
    killed_by = ""
    direction = Direction("right")
    visited = {(1,1)}

    def can_pick(self, thing):
        """Woman can only pickup friend"""
        return thing.__class__ == Friend
    
def program(percepts):
    '''Returns an action based on it's percepts'''
    #print(percepts)
    percept, current_loc, orientation, visited = percepts
    #print(percept, current_loc, orientation, visited)
    
    for p in percept[4]:
        if isinstance(p,Friend):
            print('Ín Friend location')
            return 'Pick'
    
    action = choose_action(visited,orientation,current_loc)
    return action

def choose_action(visited,orientation,current_loc):
    choice = random.choice((1,2,3,4))
    if choice == 1:
        return 'TurnRight'
    elif choice == 2:
        return 'TurnLeft'
    else:
        if agent_visited(visited,orientation,current_loc):
            return choose_action(visited,orientation,current_loc)
        else:
            return 'Forward'
    
def agent_visited(visited,orientation,current_loc):
        x,y = current_loc
        if orientation == "right":
            proposed_loc = (x + 1, y)
        elif orientation == "left":
            proposed_loc = (x - 1, y)
        elif orientation == "up":
            proposed_loc = (x, y - 1)
        elif orientation == "down":
            proposed_loc = (x, y + 1)
            
        if proposed_loc in visited:
            return True


w1 = Shopping_Mall2D(program, 6, 6)  
Rachel = Model_Based_Woman(program)
w1.add_thing(Model_Based_Woman(program), (1, 1), True)
grid = BlockGrid(w1.width, w1.height, fill=(123, 234, 123))


def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]
                

def run(steps=50, delay=1):
    """Run the Environment for given number of time steps,
        but update the GUI too."""
    for step in range(steps):
        global grid, w1
        draw_grid(w1.get_world())
        grid.show()
        if w1.is_done():
            break
        w1.step()

In [6]:
run(20)

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Model_Based_Woman decided to TurnRight at location: (1, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Model_Based_Woman decided to TurnLeft at location: (1, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


...

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Model_Based_Woman decided to move downwards at location: (3, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Death by Fire [-1000].
Agent Performance score :  0


# Goal Based Agent

<b>The main goal in our 2D world is for the agent to pick up her friend. In order to attain this goal, the agent should be alive, which ultimately means that the agent should never die by falling into the obstacle, "Fire".</b>

Similar to the Model based agent, a goal based agent will also act on a partially observable environment, but the only observable part implemented in our world, is to percieve if the proposed location to move forward, conains "Fire". If it does, the agent is prevented from moving forward to the prodposed location and chooses alternative actions. If the proposed location is safe, the agent will move forward.

This means, that the agent will act on an environment based on the current percepts and will additionally use its knowledge on considering whether the future is against the targeted goal in the environment.<br>


With this information, the intention is to prevent the agent from moving into the tile that will cause death and make the goal un-attainable.


The agent "program" function will interpret the current percept from the environment and will also percieve if the tile to be moving into, indicates "Fire". If the proposed location contains "Fire", then the agent program will propose a different action. The "execute action" function, then executes the corresponding action in the host environment.


The PEAS description for the Goal based agent is given below. The main variation of the 2D mechanics when comparing agents is in the Sensors and the Actuators part of the PEAS Description.



### PEAS Description:
* <b>Performance measure:</b> 

    Same as mentioned for the simple reflex agent.<br>
    
    
* <b>Environment:</b><br>

    Same as mentioned for the simple reflex agent.<br>
    

* <b>Actuators: </b><br>
    With respect to the orientation of the agent, she will either be able to "TurnRight" or "TurnLeft" by 90 degrees and will be able to move "Forward" in whichever direction she is currently facing. If the Forward step is towards a bump/ wall tile, the agent position doesn't change. If the agent is trapped in a tile with Fire, she is dead. If the agent is in the location of her friend, she can "Pick" her friend which is her final goal.


* <b>Sensors: </b>
  - In friend location, the agent perceives "Friend". The Goal based agent program will then initiate an action to pick up the friend. 
  - In addition to the current percept, the agent will percieve the objects present in its 4 adjacent tiles(left, right, up, down). This forms a list of the future percepts.
  - To determine if the "Forward" action is valid, the Goal based agent will look into its future percepts and determines if the proposed location to move contains "Fire". If it contains "Fire", then the move "Forward" action is considered invalid and the agent program, will be prompted for alternative action choices. If the proposed location does not contain Fire, then the move "Forward" action by the agent is considered a valid action.
  - "TurnLeft", "TurnRight" actions are executed at random.


### ADVANTAGES OF GOAL BASED AGENT
* As an added advantage to the simple reflex agent, the goal based agent will prevent itself from moving into the tile containing "Fire" thus preventing Death and ultimately moving to goal. 
* By preventing the agent's death at all instances, the average performance of the agent will increase and will never attain a score of '0'
* if the location of the goal is at a shorter distance, the performance of the aget will greatly improve.
* Such an agent behaviour can be a good rational in both single and multi agent environments.

### DISADVANTAGES OF GOAL BASED AGENT
* Although the goal based agent prevents itself from moving into the Fire, it will still be inefficient if the route to the goal follows many twists and turns in avoiding the obstacles and reaching the goal.
* The agent might take unwanted long paths, and might increase the time to reach the goal. 

Both the above disadvantages can be overcomed by using a search technique associated with it. We will look into this in the next section. 


### PERFORMANCE IN THE CURRENT 2D WORLD
The overall performance of the agent type was calcualted, by recording the performance score in the agent-environment game for 5 runs, and then by calcualting the average performance score from these 5 runs. 

<table> 
    <tr>
        <td> </td>
        <td> Run 1 <br> Performance </td>
        <td> Run 2 <br> Performance </td>
        <td> Run 3 <br> Performance </td>
        <td> Run 4 <br> Performance </td>
        <td> Run 5 <br> Performance </td>
        <td> Average performance </td>
    </tr>
    <tr>
        <td> Performance<br>score: </td>
        <td> 991 </td>
        <td> 986 </td>
        <td> 882 </td>
        <td> 833 </td>
        <td> 906 </td>
        <td> 920 </td>            
   </tr>
</table>
    
   <t><b> Average performance score : 920  </b></t>
        

### PERFORMANCE IN EXTENDED ENVIRONMENTS

If the environment size is increased, the performance of a goal based agent will decrease with it. This is because the route to the goal might get longer, although the goal can be unimately attained.

If the probability of a tile having the obstacle Fire, is increased, then the probability of the agent taking a lot of twists and turns also increases, thus decreasing the performance of the agent which respect to the number of steps it takes to reach the goal. 


In [9]:
##Goal Based agent
color = {"Fire": (255, 83, 71),
        "Friend": (240, 255, 123),
        "Heat": (230, 115, 40),
        "Goal_Based_Woman": (202, 0, 112),
        "Wall": (44, 53, 57)
        }

class Goal_Based_Woman(Agent):
    holding = []
    killed_by = ""
    direction = Direction("right")
    visited = {(1,1)}

    def can_pick(self, thing):
        """Woman can only pickup friend"""
        return thing.__class__ == Friend
    
def program(percepts):
    '''Returns an action based on it's percepts'''
    percept, current_loc, orientation, visited = percepts
    
    for p in percept[4]:
        if isinstance(p,Friend):
            print('Ín Friend location')
            return 'Pick'
    
    action = choose_action(percept,orientation)
    return action

def choose_action(percept,orientation):
    choice = random.choice((1,2,3,4))
    if choice == 1:
        return 'TurnRight'
    elif choice == 2:
        return 'TurnLeft'
    else:
        if agent_danger(percept,orientation):
            return choose_action(percept,orientation)
        else:
            return 'Forward'
    
def agent_danger(percept,orientation):
        print(percept)
        if orientation == "right" and isinstance(percept[1][0],Fire):
            return True
        elif orientation == "left" and isinstance(percept[0][0],Fire):
            return True
        elif orientation == "up" and isinstance(percept[2][0],Fire):
            return True
        elif orientation == "down" and isinstance(percept[3][0],Fire):
            return True  


w2 = Shopping_Mall2D(program, 6, 6)   
Rachel = Goal_Based_Woman(program)
w2.add_thing(Goal_Based_Woman(program), (1, 1), True)
grid = BlockGrid(w2.width, w2.height, fill=(123, 234, 123))


def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]

def run(steps=50, delay=1):
    """Run the Environment for given number of time steps,
        but update the GUI too."""
    for step in range(steps):
        global grid, w2
        draw_grid(w2.get_world())
        grid.show()
        if w2.is_done():
            break
        w2.step()    

In [10]:
run(20)

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


[[<Bump>], [None], [<Bump>], [None], [None]]
Goal_Based_Woman decided to move rightwards at location: (1, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Goal_Based_Woman decided to TurnRight at location: (2, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


...

[[<Bump>], [None], [<Bump>], [None], [None]]
Goal_Based_Woman decided to move upwards at location: (1, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Goal_Based_Woman decided to TurnRight at location: (1, 1)


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


[[<Bump>], [None], [<Bump>], [None], [None]]
Goal_Based_Woman decided to move rightwards at location: (1, 1)


# Comparison of Agent performances

From the performance scores of all the agent types, it can be concluded that the goal based agent has the highest performance in our 2D world with a score of 920 out of a possible 1000. 


Keeping the Agent alive to reach his goal, has added to increase in the performance score of the goal based agent. The limitations of the model based agent and the simple refex agent has proved it to be incapable in the current world.

# Assignment Part 2 - Search the world

# Problem Solving Using Search


The current problem in hand is a 2D grid routing problem, which is for the 2D world that was designed in the previous section.

Considering the goal based agent from the above section is equipped with a search algorithm, the impact of three different uninformed search strategies was studied in the instances of the same Environment.


<b>In order to study the impact of the search strategies, our grid size was increased to a 10x10 grid problem. This can be extended to any random grid size.</b>


The initial state in our world is the position [1,1] in the grid, where the agent will start the route. The goal state can be fixed at any [x,y] location in the grid. In the current world, the goal is set at location [8,8]. The "Fire" obstacles, was set at random with a uniform probability of 0.2. 


In certain cases, the location of "Fire" and "Friend" can be in location [8,8], which is unavoidable in a randomized environment.


The code in the next cell for the GridProblem class, contains 
- the initial and the goal states defined. It can take any randomised values for the same. 
- The directions variable describes the possible directions the agent will move and is used in calculating if any obsacles are present in the proposed direction. 
- The action cost function is used to calculate the cost of each path that will be taken by the agent to reach the goal. It is the straight line distance between the tiles.
- The actions defines the directions the agent can move
- The result function defines if the proposed location contains obtacles and prevents the agent from moving towards it.
- The goal test function is_goal is included in the Assignment supporting file, and is used to test if the agent has reached the goal.


The three uninformed search strategies that will be used for the current problem are:
* Breadth First search
* Depth Limited search
* Iterative Deepening Search

In [11]:

############################ All the below code was used from the file search4e.ipynb######### 
    
class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, obstacles = (), **kwds):
        self.__dict__.update(initial=initial, goal=goal, obstacles = obstacles, **kwds)

    def actions(self, state):        raise NotImplementedError

    def result(self, state, action): raise NotImplementedError

    def is_goal(self, state):        return state == self.goal

    def action_cost(self, s, a, s1): return 1

    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)


class Node:
    "A Node in a search tree."

    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)

    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))

    def __lt__(self, other): return self.path_cost < other.path_cost


failure = Node('failure', path_cost=math.inf)  # Indicates an algorithm couldn't find a solution.
cutoff = Node('cutoff', path_cost=math.inf)  # Indicates iterative deepening search was cut off.


def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5


FIFOQueue = deque

LIFOQueue = list


class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)



def g(n): return n.path_cost



def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)


class GridProblem(Problem):
    """Finding a path on a 2D grid with obstacles. Obstacles are (x, y) cells."""

    def __init__(self, initial=(1, 1), goal=(8, 8), obstacles= (), **kwds):
        Problem.__init__(self, initial=initial, goal=goal, obstacles = obstacles, **kwds)

    directions = [          (0, -1), 
                  (-1, 0) ,          (1,  0),
                            (0, +1),           ]
    
    def action_cost(self, s, action, s1): return straight_line_distance(s, s1)
                  
    def result(self, state, action): 
        "Both states and actions are represented by (x, y) pairs."
        return action if action not in self.obstacles else state
    
    def actions(self, state):
        """You can move one cell in any of `directions` to a non-obstacle cell."""
        x, y = state
        return {(x + dx, y + dy) for (dx, dy) in self.directions} - self.obstacles

# Extended 2D Environment for the Search strategies

In [12]:
class Shopping_Mall2D_search(XYEnvironment):

    fire_probability = 0.2  # Probability to spawn fire in a location. (From Chapter 7.2)

    def __init__(self, agent_program, width=5, height=5):
        super().__init__(width, height)
        goal = []
        obstacles = []
        self.visible = False
        self.init_world(agent_program)

    def init_world(self, program):
        """Spawn items in the world based on probabilities"""

        "WALLS"
        self.add_walls()

        "FIRE"
        for x in range(self.x_start, self.x_end):
            for y in range(self.y_start, self.y_end):
                if (x == 1) & (y == 1):
                    continue
                if random.random() < self.fire_probability:
                    
                    self.add_thing(Fire(), (x, y), True)
                    self.add_thing(Heat(), (x - 1, y), True)
                    self.add_thing(Heat(), (x, y - 1), True)
                    self.add_thing(Heat(), (x + 1, y), True)
                    self.add_thing(Heat(), (x, y + 1), True)


    def get_world(self, show_walls=True):
        """Return the items in the world"""
        obstacles = []
        goal = []
        result = []
        x_start, y_start = (0, 0) if show_walls else (1, 1)

        if show_walls:
            x_end, y_end = self.width, self.height
        else:
            x_end, y_end = self.width - 1, self.height - 1

        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                things_xy = self.list_things_at((x, y))
                row.append(things_xy)
                if len(things_xy):
                    for i in range(0,len(things_xy)):
                        if isinstance(things_xy[i], Friend):
                            goal.append((x,y))
                        elif isinstance(things_xy[i], Fire):
                            obstacles.append((x,y))
                        elif isinstance(things_xy[i], Wall):
                            obstacles.append((x,y))
            result.append(row)
        return (result,obstacles,goal)
    
    def percepts_from(self, agent, location, tclass=Thing):
        """Return percepts from a given location,
        and replaces some items with percepts from chapter 7."""
        thing_percepts = {
            Friend: Friend(),
            Wall: Bump(),
            Fire: Fire(),
            Heat: Heat()}

        """Agents don't need to get their percepts"""
        thing_percepts[agent.__class__] = None

        result = [thing_percepts.get(thing.__class__, thing) for thing in self.things
                  if thing.location == location and isinstance(thing, tclass)]
        return result if len(result) else [None]

    def percept(self, agent,path):
        """Return things in adjacent (not diagonal) cells of the agent.
        Result format: [Left, Right, Up, Down, Center / Current location]"""
        x, y = agent.location
        result = []
        result.append(self.percepts_from(agent, (x - 1, y)))
        result.append(self.percepts_from(agent, (x + 1, y)))
        result.append(self.percepts_from(agent, (x, y - 1)))
        result.append(self.percepts_from(agent, (x, y + 1)))
        result.append(self.percepts_from(agent, (x, y)))
        
        return (result,agent.location,agent.direction.direction,path)
    
    def execute_action(self, agent, action):
        """Modify the state of the environment based on the agent's actions.
        Performance score taken directly out of the book."""

        if self.in_danger(agent):
            return
  
        agent.bump = False
        if action == 'TurnRight':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.direction += Direction.R
            agent.performance -= 1
        elif action == 'TurnLeft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.direction += Direction.L
            agent.performance -= 1
        elif action == 'Forward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
            agent.performance -= 1
        elif action == 'Pick':
            things = [thing for thing in self.list_things_at(agent.location)
                      if agent.can_pick(thing)]
            if len(things):
                print("Picking up", things[0].__class__.__name__)
                if len(things):
                    agent.holding.append(things[0])
                    self.delete_thing(things[0])
                    agent.performance += 1000
                    print('Agent Performance score : ',agent.performance)


    def in_danger(self, agent):
        """Check if Rachel is in danger (Fire), if she is, kill her"""
        for thing in self.list_things_at(agent.location):
            if isinstance(thing, Fire):
                agent.alive = False
                agent.performance -= 1000
                agent.killed_by = thing.__class__.__name__
                return True
        return False

    def is_done(self):
        """The game is over when Rachel is killed
        or if she picks up her friend."""
        #woman = [agent for agent in self.agents if isinstance(agent, Woman)]
        #if woman[0].alive:
        if any(agent.is_alive() for agent in self.agents):
            goal_completed = not any(isinstance(thing, Friend) for thing in self.things)
            if goal_completed:
                print("Woman completed her goal{}."
                  .format(" with Friend [+1000]!" if Friend() not in self.things else " without Friend [+0]"))
                return True
            else:
                return False
        else:
            dead_agents = not any(agent.is_alive() for agent in self.agents)
            print("Death by Fire [-1000].")
            print('Agent Performance score : ',0)
            return True

    def update(self, delay=1):
        sleep(delay)
        if self.visible:
            self.conceal()
            self.reveal()
        else:
            self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_grid(w.get_world())
        self.grid.show()
        self.visible = True

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))
        
    def step(self,path):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent,path)))
                else:
                    actions.append("")
            actions = list(chain.from_iterable(actions))
            move = list(zip(cycle(self.agents), actions))
            for (agent, action) in list(move):
                self.execute_action(agent, action)
            self.exogenous_change()
    
    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end-1),
                    random.randint(self.y_start, self.y_end-1))
        if exclude is not None:
            while(location == exclude):
                location = (random.randint(self.x_start, self.x_end-1),
                            random.randint(self.y_start, self.y_end-1))
        return location


The agent program function given below, will take the sequence of the steps in the path that was determined by the search strategies and will execute the agent to traverse the route to the goal/destination. The performance of the agent is displayed once it reaches the goal.

In [13]:

def program(percepts):
    '''Returns an action based on it's percepts'''

    actions = []
    percept, current_loc, orientation, path = percepts
    
    for p in percept[4]:
        if isinstance(p,Friend):
            print('Ín Friend location')
            actions.append('Pick')
            return actions
    
    proposed_loc = path.pop(0)
    move = [i1 - i2 for (i1,i2) in zip(proposed_loc,list(current_loc))]
    
    if move == [0,-1]:
        if orientation == 'right':
            actions.append('TurnLeft')
            actions.append('Forward')
        if orientation == 'left':
            actions.append('TurnRight')
            actions.append('Forward')
        if orientation == 'up':
            actions.append('Forward')
        if orientation == 'down':
            actions.append('TurnRight')
            actions.append('TurnRight')
            actions.append('Forward')
    
    if move == [0,+1]:
        if orientation == 'right':
            actions.append('TurnRight')
            actions.append('Forward')
        if orientation == 'left':
            actions.append('TurnLeft')
            actions.append('Forward')
        if orientation == 'up':
            actions.append('TurnRight')
            actions.append('TurnRight')
            actions.append('Forward')
        if orientation == 'down':
            actions.append('Forward')
            
    if move == [-1,0]:
        if orientation == 'right':
            actions.append('TurnRight')
            actions.append('TurnRight')
            actions.append('Forward')
        if orientation == 'left':
            actions.append('Forward')
        if orientation == 'up':
            actions.append('TurnLeft')
            actions.append('Forward')
        if orientation == 'down':
            actions.append('TurnRight')
            actions.append('Forward')

    if move == [1,0]:
        if orientation == 'right':
            actions.append('Forward')
        if orientation == 'left':
            actions.append('TurnRight')
            actions.append('TurnRight')
            actions.append('Forward')
        if orientation == 'up':
            actions.append('TurnRight')
            actions.append('Forward')
        if orientation == 'down':
            actions.append('TurnLeft')            
            actions.append('Forward') 
            
    return actions



### Breadth First Search

In the breadth first search technique, the root node is initially expanded, following which all its successors in the next level are expanded, and this continues until every level of successors are expanded. The expansion moves to the next level/depth only after the all the nodes at the current level/depth are expanded.

From the current 2D world, we generate the obstacles(Fire) in the world at random positions. All the location of the obstacles and the location of the goal is passed to the Grid problem class to formulate the problem with the given initial and goal states.

The Breadth first search technique is then implemented, which identifies the path in the grid as it expands the nodes in every step it moves in the grid. The directions variable in the Grid problem specifies the possible nodes the search can expand to from each node/tile. The shallowest unexpanded step in the grid is considered first. This is implemented using the First-in, first-out logic as used by a queue. As the nodes are expanded, the goal state is checked at step. If the goal is reached, the path states or the sequence of the [x,y] coordinates that were follwed to reach the goal is retrieved.


The sequence of steps resulting from the breadth first search was then implemented in our 2D world as a step by step path that agent takes to reach the goal.


The code for the search algorithm was used from the AIMA file search4e.ipynb


In [14]:
### The code for the search algorithm was used from the AIMA file search4e.ipynb
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


color = {"Fire": (255, 83, 71),
        "Friend": (240, 255, 123),
        "Heat": (230, 115, 40),
        "Woman": (202, 0, 112),
        "Wall": (44, 53, 57)
        }

class Woman(Agent):
    holding = []
    killed_by = ""
    direction = Direction("right")

    def can_pick(self, thing):
        """Woman can only pickup friend"""
        return thing.__class__ == Friend
    
w4 = Shopping_Mall2D_search(program,10, 10)
Rachel = Woman(program)
w4.add_thing(Rachel, (1, 1), True)
w4.add_thing(Friend(), (8,8), True)
grid = BlockGrid(w4.width, w4.height, fill=(123, 234, 123))
results = w4.get_world()
obstacles = set(results[1])
goal = set(results[2])
obstacles -= goal
s1 = GridProblem(initial = (1,1), goal = goal.pop(), obstacles = obstacles)

BFS = path_states(breadth_first_search(s1))
path1 = list(BFS)

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]              

def run(steps=50, path = None, delay=1):
    """Run the Environment for given number of time steps,
        but update the GUI too."""
    for step in range(steps):
        global grid, w4
        draw_grid(w4.get_world()[0])
        grid.show()
        if w4.is_done():
            break
        w4.step(path)    
        

if len(path1) > 1:
    path = [list(x) for x in path1][1:]
else:
    path = path1

    
print("\n")
print("Path from Breadth First search")
print(path)



Path from Breadth First search
[[1, 2], [1, 3], [2, 3], [2, 4], [2, 5], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6], [7, 6], [8, 6], [8, 7], [8, 8]]


The sequence of the path steps, for the agent to take from the initial position to the goal is displayed as an ouput in the previous cell. The below cell use the path obtained in the world to make the agent reach the goal.

In [15]:
if(path == []):
    print('Agent is surrounded by fire in the Initial position')
    print('Rerun the entire search experiment to generate new random obstacles(Fire)')
else:
    run(20,path)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman decided to TurnRight at location: (1, 1)
Woman decided to move downwards at location: (1, 1)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman decided to move downwards at location: (1, 2)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


...

Woman decided to move downwards at location: (8, 7)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Ín Friend location
Picking up Friend
Agent Performance score :  981


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman completed her goal with Friend [+1000]!.


### Depth Limited Search
As opposing to the Breadth first search, the depth first search and the depth limited search, will pass through the deepest nodes first instead of the shallow nodes in the same level/depth. The disadvantage of the depth first search is that the limit of how deep the graph is expanded depends on the maximum depth available for the problem. Due to this, the nodes of the deepest level might be expanded even when a shallowest depth solution is available. To overcome this, the depth limited search specifies the depth limit until which the search can expand.


In our current problem, the depth limit is set as 10, which means the nodes are expended until a depth of 10, to find the goal. The directions variable in the Grid problem specifies the possible nodes the search can expand to from each node/tile in the grid. As the size of the grid environment is increased, the depth should also be increased for the problem to reach its completion state.


The resulting sequence of paths is then provided as an input to the 2D world to implement a step-by-step path that the agent can take to reach the goal.

The code for the search algorithm was used from the AIMA file search4e.ipynb

In [16]:
### The code for the search algorithm was used from the AIMA file search4e.ipynb
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

w4.delete_thing(Rachel)

class Woman(Agent):
    holding = []
    killed_by = ""
    direction = Direction("right")

    def can_pick(self, thing):
        """Woman can only pickup friend"""
        return thing.__class__ == Friend

Rachel = Woman(program)
w4.add_thing(Rachel, (1, 1), True)
w4.add_thing(Friend(), (8,8), True)

DLS = path_states(depth_limited_search(s1,limit = 20))
path3 = list(DLS)

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]              

def run(steps=50, path = path, delay=1):
    """Run the Environment for given number of time steps,
        but update the GUI too."""
    for step in range(steps):
        global grid, w4
        draw_grid(w4.get_world()[0])
        grid.show()
        if w4.is_done():
            break
        w4.step(path)   
        
if len(path3) > 1:
    path = [list(x) for x in path3][1:]
else:
    path = path3

print("\n")
print("Path from Depth limited search")
print(path)

list.remove(x): x not in list
  in Environment delete_thing
  Thing to be removed: <Friend> at (8, 8)
  from list: [(<Wall>, (0, 0)), (<Wall>, (0, 9)), (<Wall>, (1, 0)), (<Wall>, (1, 9)), (<Wall>, (2, 0)), (<Wall>, (2, 9)), (<Wall>, (3, 0)), (<Wall>, (3, 9)), (<Wall>, (4, 0)), (<Wall>, (4, 9)), (<Wall>, (5, 0)), (<Wall>, (5, 9)), (<Wall>, (6, 0)), (<Wall>, (6, 9)), (<Wall>, (7, 0)), (<Wall>, (7, 9)), (<Wall>, (8, 0)), (<Wall>, (8, 9)), (<Wall>, (9, 0)), (<Wall>, (9, 9)), (<Wall>, (0, 1)), (<Wall>, (9, 1)), (<Wall>, (0, 2)), (<Wall>, (9, 2)), (<Wall>, (0, 3)), (<Wall>, (9, 3)), (<Wall>, (0, 4)), (<Wall>, (9, 4)), (<Wall>, (0, 5)), (<Wall>, (9, 5)), (<Wall>, (0, 6)), (<Wall>, (9, 6)), (<Wall>, (0, 7)), (<Wall>, (9, 7)), (<Wall>, (0, 8)), (<Wall>, (9, 8)), (<Fire>, (1, 5)), (<Heat>, (1, 4)), (<Heat>, (2, 5)), (<Heat>, (1, 6)), (<Fire>, (1, 7)), (<Heat>, (2, 7)), (<Heat>, (1, 8)), (<Fire>, (2, 2)), (<Heat>, (1, 2)), (<Heat>, (2, 1)), (<Heat>, (3, 2)), (<Heat>, (2, 3)), (<Fire>, (4, 5)), (<

The sequence of steps from the above resulting path is implemented in our 2D world in order to assess the performance of using the depth limited search for the agent to reach the goal.

<i>Note: The warnings generated by the above cell output can be ignored</i>

In [17]:
if(path == []):
    print('Agent is surrounded by fire in the Initial position')
    print('Rerun the entire search experiment to generate new random obstacles(Fire)')
else:
    run(30,path)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman decided to move rightwards at location: (1, 1)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman decided to move rightwards at location: (2, 1)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


...

Woman decided to move rightwards at location: (7, 8)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Ín Friend location
Picking up Friend
Agent Performance score :  972


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman completed her goal with Friend [+1000]!.


### Iterative Deepening Search

The iterative deepening search is similar to the depth limited search, but it is not required to set the depth limit here as it is gradually increased by the search algorithm until the goal is reached. The Iterative deepening search is a combination of both the breadth first search and the depth limited search as it tries to iteratively apply the depth limited search as limit increases to reach the goal. It tries to find the goal in the shallowest level and if not successful it goes to the next level/depth as a new iteration of the explored levels and the newly unexplored nodes of the next level. 

Hence, in our current problem, the performance of the Iterative deepening search will be closer to the performance of the breadth first search algorithm with minor differences that is added due to the iterative peformance.

The code for the search algorithm was used from the AIMA file search4e.ipynb

In [18]:
### The code for the search algorithm was used from the AIMA file search4e.ipynb
def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
w4.delete_thing(Rachel)

class Woman(Agent):
    holding = []
    killed_by = ""
    direction = Direction("right")

    def can_pick(self, thing):
        """Woman can only pickup friend"""
        return thing.__class__ == Friend

Rachel = Woman(program)
w4.add_thing(Rachel, (1, 1), True)
w4.add_thing(Friend(), (8,8), True)

Iterative_deepening_DFS = path_states(iterative_deepening_search(s1))
path2 = list(Iterative_deepening_DFS)


def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]              

def run(steps=50, path = path, delay=1):
    """Run the Environment for given number of time steps,
        but update the GUI too."""
    for step in range(steps):
        global grid, w4
        draw_grid(w4.get_world()[0])
        grid.show()
        if w4.is_done():
            break
        w4.step(path)   
        
if len(path2) > 1:
    path = [list(x) for x in path2][1:]
else:
    path = path2

print("\n")
print("Path from Iterative Deepening search")
print(path)

list.remove(x): x not in list
  in Environment delete_thing
  Thing to be removed: <Friend> at (8, 8)
  from list: [(<Wall>, (0, 0)), (<Wall>, (0, 9)), (<Wall>, (1, 0)), (<Wall>, (1, 9)), (<Wall>, (2, 0)), (<Wall>, (2, 9)), (<Wall>, (3, 0)), (<Wall>, (3, 9)), (<Wall>, (4, 0)), (<Wall>, (4, 9)), (<Wall>, (5, 0)), (<Wall>, (5, 9)), (<Wall>, (6, 0)), (<Wall>, (6, 9)), (<Wall>, (7, 0)), (<Wall>, (7, 9)), (<Wall>, (8, 0)), (<Wall>, (8, 9)), (<Wall>, (9, 0)), (<Wall>, (9, 9)), (<Wall>, (0, 1)), (<Wall>, (9, 1)), (<Wall>, (0, 2)), (<Wall>, (9, 2)), (<Wall>, (0, 3)), (<Wall>, (9, 3)), (<Wall>, (0, 4)), (<Wall>, (9, 4)), (<Wall>, (0, 5)), (<Wall>, (9, 5)), (<Wall>, (0, 6)), (<Wall>, (9, 6)), (<Wall>, (0, 7)), (<Wall>, (9, 7)), (<Wall>, (0, 8)), (<Wall>, (9, 8)), (<Fire>, (1, 5)), (<Heat>, (1, 4)), (<Heat>, (2, 5)), (<Heat>, (1, 6)), (<Fire>, (1, 7)), (<Heat>, (2, 7)), (<Heat>, (1, 8)), (<Fire>, (2, 2)), (<Heat>, (1, 2)), (<Heat>, (2, 1)), (<Heat>, (3, 2)), (<Heat>, (2, 3)), (<Fire>, (4, 5)), (<

The sequence of steps from the above resulting path is implemented in our 2D world in order to assess the performance of using the Iterative Deepening search for the agent to reach the goal.

<i>Note: The warnings generated by the above cell output can be ignored</i>

In [19]:
if(path == []):
    print('Agent is surrounded by fire in the Initial position')
    print('Rerun the entire search experiment to generate new random obstacles(Fire)')
else:
    run(20,path)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman decided to move rightwards at location: (1, 1)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman decided to move rightwards at location: (2, 1)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


...

Woman decided to move rightwards at location: (7, 8)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Ín Friend location
Picking up Friend
Agent Performance score :  982


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Woman completed her goal with Friend [+1000]!.


The performance of the agents in all the three search environments were recorded for 5 executions. The overall performance of a particular search startegy is the average of the 5 execution scores. The results are gievn below.

<table>
    <tr>
        <td><b> Search Type </b></td>
        <td><b>Run 1</b></td>
        <td><b>Run 2</b></td>
        <td><b>Run 3</b></td>
        <td><b>Run 4</b></td>
        <td><b>Run 5</b></td>
        <td><b>Average Performance </b></td>
    </tr>
    <tr>
        <td><b> Breadth First Search </b></td>
        <td> 981 </td>
        <td> 980 </td>
        <td> 979 </td>
        <td> 975 </td>
        <td> 981 </td>
        <td> 979 </td>
    </tr>
        <tr>
            <td> <b>Depth Limited Search </b></td>
        <td> 969 </td>
        <td> 970 </td>
        <td> 967 </td>
        <td> 979 </td>
        <td> 973 </td>
        <td> 972 </td>
    </tr>
     <tr>
         <td><b>Iterative deepening search</b> </td>
        <td> 983 </td>
        <td> 976 </td>
        <td> 977 </td>
        <td> 983 </td>
        <td> 983 </td>
        <td> 980 </td>
    </tr>
</table>


From the performance results shown above, it can be seen that the Iterative Deepening search has the highest perforance when compared to the remaining search startegies. Although it is similar to the Breadth first search, the iterative factor of the search strategy has given a minor boost in terms of the increased performance score.

# Assignment Part 3 - Forward Chaining and Backward Chaining

Forward chaining is a reasoning method that can be applied to a Knowledge base starting from the facts that are known and extracting more data from it to reach a solution/goal. Backward chaining is the vice-versa operation, where it starts from the goal to reach the known facts that are in support of the goal.

For the 2D world in discussion, the reasoning methods would be helpful in keeping the agent alive in the game and to reach its goal. 

Let us assume to be using a goal based agent for the current problem with percepts from the four adjacent squares and the current location. The known facts in our world is that the agent is in the initial location [1,1] and is safe as there is no indication of "Heat". 

<b> Consider the question in hand is "Does the location [2,1] contain Fire?".</b>

Forward chaining would be helpful in reasoning the above question, as to whether "Fire" is located at a particular tile. The indication of "Heat" around the tiles containing "Fire", would be the inferences that would be used in reasoning the presence of "Fire" in a particular location. If [2,1] contains "Fire", the current location [1,1] should indicate "Heat", which is not True and hence it can be reasoned that [2,1] is safe to move. This reasoning follows as a chain until the path to the goal is reached.


Backward chaining can be used to traverse backward from the query to reach the known facts. Considering that "Fire" is present in a particular location, it can be deduced that its adjacent squares contain "Heat" and further chaining thru the remaining tiles, to indicate that, if no "Heat" nor "Fire", then the location is safe.

In both the methods, the agent gradually comes to a gaurenteed conclusion based on the information that it starts with.


# References
[Reference 1] - Artificial Intelligence: A Modern Approach(3rd edition), Stuart Russel, Peter Norvig<br>

The code for the three search algorithms was implemented from the search4e.ipynb in the AIMA repositary.

All the remaining code in the current jupyter notebook file were partially referenced from the agents.ipynb and agents.py inthe AIMA repository with/without some additional changes specific to our own 2D world.
